In [1]:
from __future__ import print_function
import json
from pymongo import MongoClient
import numpy as np
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
from bs4 import BeautifulSoup as bs
import requests
import csv
import tqdm
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from geotext import GeoText
from googletrans import Translator

# Leggo file tweet_finali

In [2]:
tweet_finali= pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\csv_finali\\final_totali.csv",
               engine="python", encoding="utf-8")
tweet_finali.head()

,id,date,time,device,place_user,text,hashtag,score,FGA_GSW,FGA_TOR,...,PERSONAL_GSW,PERSONAL_TOR,TECHNICAL_GSW,TECHNICAL_TOR,retweet_id,timestat,quarter,sentiment,team,gara
0,1134248118231416833,2019-05-31,00:00:04,Twitter for iPhone,Tehran,Let’s fucking go #DubNation,['dubnation'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000,Golden State Warriors,1
1,1134248122291560448,2019-05-31,00:00:05,TweetDeck,United States,Draymond Green has a 69.2% chance to see more ...,"['nbaplayoffs', 'strengthinnumbers', 'nbafinals']",0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.9246,Golden State Warriors,1
2,1134248146668851200,2019-05-31,00:00:11,Twitter for iPad,"Washington, DC",Steph Curry walks into Scotiabank Arena ahead ...,['strengthinnumbers'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.134243e+18,NaN,0,0.0000,Golden State Warriors,1
3,1134248155149754369,2019-05-31,00:00:13,Twitter for iPhone,"Noisy-le-Roi, France",Steph Curry walks into Scotiabank Arena ahead ...,['strengthinnumbers'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.134243e+18,NaN,0,0.0000,Golden State Warriors,1
4,1134248161931751425,2019-05-31,00:00:14,Twitter for Android,"San Diego, CA",I'm shimmying for #TeamSteph! Do you think Ste...,"['nbaplayoffs', 'dubnation', 'nbafinals', 'tea...",0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.5411,Golden State Warriors,1


# Conto numero tweet per gara

In [36]:
tweetGara=tweet_finali.groupby("gara").count()
tweetGara=tweetGara.reset_index()
tweetGara=tweetGara[["gara","id"]]
tweetGara.columns=["Game","#Tweet"]
tweetGara

,Game,#Tweet
0,1,226103
1,2,153897
2,3,139553
3,4,165585
4,5,258618
5,6,362033


# Prendo tuttti i tweet di una gara

In [4]:
tweetGara=tweet_finali[tweet_finali["gara"]==6]
tweetGara.gara.unique()
tweetGara.head()

,id,date,time,device,place_user,text,hashtag,score,FGA_GSW,FGA_TOR,...,PERSONAL_GSW,PERSONAL_TOR,TECHNICAL_GSW,TECHNICAL_TOR,retweet_id,timestat,quarter,sentiment,team,gara
83717,1139321556264464390,2019-06-14,00:00:06,Twitter for Android,NaN,KD my brother❤\n\n#DubNation \n#OneLastDance 💃...,"['kdstrong', 'dubnation', 'onelastdance', 'nba...",0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.139291e+18,NaN,0,0.3612,Golden State Warriors,6
83718,1139321558860591105,2019-06-14,00:00:07,NBA PLOW,Internet,Oakland hip-hop legends to perform last Warrio...,['goldenstatewarriors'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000,Golden State Warriors,6
83719,1139321563344281600,2019-06-14,00:00:08,San Francisco PLOW,Internet,Oakland hip-hop legends to perform last Warrio...,['goldenstatewarriors'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000,Golden State Warriors,6
83720,1139321569170403328,2019-06-14,00:00:09,Twitter for iPhone,"Dallas, TX","For Oakland, for KD 🙏\n\n#StrengthInNumbers ht...",['strengthinnumbers'],0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.139321e+18,NaN,0,0.0000,Golden State Warriors,6
83721,1139321571531808768,2019-06-14,00:00:10,Statusbrew,NaN,Stephen Curry to take lead in absence of Kevin...,"['strengthinnumbers', 'nbafinals']",0 - 0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0000,Golden State Warriors,6


# Salvo file tweet gara x (operazione effettuata per ogni gara)

In [8]:
tweetGara=tweetGara[["id","place_user","team"]]
tweetGara.to_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\gara6\\posizioneTeam_gara6.csv",encoding="utf-8",
                 index=False)

# Ottengo il contenuto place_user tweet più lungo

In [9]:
tweetGara["place_user"]

field_length = tweetGara.place_user.astype(str).map(len)
print(tweetGara.loc[field_length.idxmax(), 'place_user'])

USA. UK. Israel. France. Swiss. Italy. Lithuania. Poland. Germany. Austria. Hungary. Belgium. Bulgaria. Turkey. Slovakia. Czech Republic. Ukraine.


In [10]:
#type(tweetGara1["place_user"][3])
tweetGara["place_user"]=tweetGara["place_user"].astype(str)
contaVirgole=tweetGara[["id","place_user"]]
contaVirgole["place_user"]=contaVirgole["place_user"].apply(lambda stringa: stringa.count(","))
index=contaVirgole["place_user"].idxmax()
contaVirgole.loc[index]["id"]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


1139381083718291456

In [11]:
contaVirgole.groupby("place_user").count().reset_index()

,place_user,id
0,0,224172
1,1,131581
2,2,5983
3,3,217
4,4,33
5,5,38
6,6,4
7,7,2
8,9,3


In [12]:
ris=contaVirgole[contaVirgole["place_user"]==5]["id"]
id_ris=ris.values
tweetGara[tweetGara["id"].isin(id_ris)]

,id,place_user,team
104992,1139380781506158593,"twd, got, ahs, lcdp, bm, pb",Golden State Warriors
498061,1139335884023750657,"Toronto, Montreal, Calgary, Vancouver, Halifax...",Toronto Raptors
522716,1139359589009412097,La vida se encarga de poner cada cosa en su lu...,Toronto Raptors
555819,1139379229827534851,"loona,bts,gwsn,rv,bp,twice",Toronto Raptors
570620,1139380903715602437,"loona, d(ioi)/dx, bp, rv, exid, clc",Toronto Raptors
589900,1139383299900506112,"loona,bts,gwsn,rv,bp,twice",Toronto Raptors
594180,1139383772518895627,"loona, d(ioi)/dx, bp, rv, exid, clc",Toronto Raptors
608048,1139385352953556997,"North Etobicoke, Woodbridge, Maple, Bolton, No...",Toronto Raptors
613270,1139385950235066370,"loona, d(ioi)/dx, bp, rv, exid, clc",Toronto Raptors
624883,1139387279992950784,"LA, NYC, LDN, RDU, TOR, CBD...",Toronto Raptors


# Sistemo posizione place tweet gara x (operazione effettuata per ogni gara)

In [7]:
gara_tweet=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\gara6\\posizioneTeam_gara6.csv",
                 engine="python",encoding="utf-8")
gara_tweet.head()

,id,place_user,team
0,1139321556264464390,NaN,Golden State Warriors
1,1139321558860591105,Internet,Golden State Warriors
2,1139321563344281600,Internet,Golden State Warriors
3,1139321569170403328,"Dallas, TX",Golden State Warriors
4,1139321571531808768,NaN,Golden State Warriors


In [14]:
len(gara_tweet)

362033

# Carico csv utili per sistemare le posizioni

In [3]:
legendaUS=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\citiesUS.csv",
                    engine="python", encoding="utf-8")
legendaCA=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\citiesCA.csv",
                    engine="python", encoding="utf-8")
stati_code=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\countries_codes.csv",
                    engine="python", encoding="utf-8")
cities_world=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\worldcities.csv",
                        engine="python",encoding="utf-8")

In [9]:
stati_code.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,"""AF""","""AFG""","""4""","""33""","""65"""
1,Albania,"""AL""","""ALB""","""8""","""41""","""20"""
2,Algeria,"""DZ""","""DZA""","""12""","""28""","""3"""
3,American Samoa,"""AS""","""ASM""","""16""","""-14.3333""","""-170"""
4,Andorra,"""AD""","""AND""","""20""","""42.5""","""1.6"""


In [13]:
legendaUS.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


In [14]:
legendaCA.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322


In [15]:
cities_world.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Malishevë,Malisheve,42.4822,20.7458,Kosovo,XK,XKS,Malishevë,admin,NaN,1901597212
1,Prizren,Prizren,42.2139,20.7397,Kosovo,XK,XKS,Prizren,admin,NaN,1901360309
2,Zubin Potok,Zubin Potok,42.9144,20.6897,Kosovo,XK,XKS,Zubin Potok,admin,NaN,1901608808
3,Kamenicë,Kamenice,42.5781,21.5803,Kosovo,XK,XKS,Kamenicë,admin,NaN,1901851592
4,Viti,Viti,42.3214,21.3583,Kosovo,XK,XKS,Viti,admin,NaN,1901328795


In [12]:
legendaCA[["city","admin","iso2","population"]].head()

,city,admin,iso2,population
0,Toronto,Ontario,CA,5213000
1,Montréal,Québec,CA,3678000
2,Vancouver,British Columbia,CA,2313328
3,Ottawa,Ontario,CA,1145000
4,Calgary,Alberta,CA,1110000


In [14]:
cities_world[["city","country","iso3","iso2","admin_name","population"]].head()

,city,country,iso3,iso2,admin_name,population
0,Malishevë,Kosovo,XKS,XK,Malishevë,NaN
1,Prizren,Kosovo,XKS,XK,Prizren,NaN
2,Zubin Potok,Kosovo,XKS,XK,Zubin Potok,NaN
3,Kamenicë,Kosovo,XKS,XK,Kamenicë,NaN
4,Viti,Kosovo,XKS,XK,Viti,NaN


In [15]:
cities_world=cities_world[["city","country","iso3","iso2","admin_name","population","capital"]]
cities_world.columns=["city","country","iso3","iso2","state_name","population","capital"]
cities_world.head()

,city,country,iso3,iso2,state_name,population,capital
0,Malishevë,Kosovo,XKS,XK,Malishevë,NaN,admin
1,Prizren,Kosovo,XKS,XK,Prizren,NaN,admin
2,Zubin Potok,Kosovo,XKS,XK,Zubin Potok,NaN,admin
3,Kamenicë,Kosovo,XKS,XK,Kamenicë,NaN,admin
4,Viti,Kosovo,XKS,XK,Viti,NaN,admin


In [16]:
len(cities_world)

12959

In [17]:
cities_world["population"]=cities_world["population"].fillna(0)
capitali_world=cities_world[cities_world["capital"]=="primary"]

In [18]:
capitali_world=capitali_world.loc[capitali_world.groupby("country")["population"].idxmax()]

# Carico df ottenuti nel notebook "continuazione_sistemarePosizione"

In [16]:
%store -r statiCA_capoluogo
statiCA_capoluogo=statiCA_capoluogo[["city","state","country"]]

In [17]:
%store -r statiUS_capoluogo
statiUS_capoluogo.head()

,city,state_id,state_name,lat,lng,population,iso3
19305,Birmingham,AL,Alabama,33.5276,-86.7988,744189.0,USA
18546,Anchorage,AK,Alaska,61.1508,-149.1091,253421.0,USA
26421,Phoenix,AZ,Arizona,33.5722,-112.0891,4081849.0,USA
20024,Little Rock,AR,Arkansas,34.7255,-92.3581,442699.0,USA
16938,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA


In [21]:
stati_code=stati_code[["Country","Alpha-3 code","Alpha-2 code"]]
valori3=stati_code["Alpha-3 code"].apply(lambda iso: iso[2:5])
valori2=stati_code["Alpha-2 code"].apply(lambda iso: iso[2:4])
stati_code["iso3"]=valori3
stati_code["iso2"]=valori2
stati_code=stati_code[["Country","iso3","iso2"]]
stati_code.head()

,Country,iso3,iso2
0,Afghanistan,AFG,AF
1,Albania,ALB,AL
2,Algeria,DZA,DZ
3,American Samoa,ASM,AS
4,Andorra,AND,AD


In [22]:
stati_code_noUSACAN=stati_code[(stati_code["iso3"]!="USA") & (stati_code["iso3"]!="CAN")]

In [23]:
legendaUS=legendaUS[["city","state_id","state_name","population"]]
legendaUS["iso3"]="USA"
legendaUS["population"]=legendaUS["population"].fillna(0)
legendaUS.head()

,city,state_id,state_name,population,iso3
0,Prairie Ridge,WA,Washington,0.0,USA
1,Edison,WA,Washington,0.0,USA
2,Packwood,WA,Washington,0.0,USA
3,Wautauga Beach,WA,Washington,0.0,USA
4,Harper,WA,Washington,0.0,USA


In [148]:
legendaCA.head()

,city,state_name,iso3,population
0,Toronto,Ontario,CAN,5213000
1,Montréal,Québec,CAN,3678000
2,Vancouver,British Columbia,CAN,2313328
3,Ottawa,Ontario,CAN,1145000
4,Calgary,Alberta,CAN,1110000


In [25]:
stati_US=legendaUS[["state_id","state_name"]]
stati_US["iso3"]="USA"
stati_US=stati_US.drop_duplicates()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
cities_world_stateUSA=pd.merge(cities_world,stati_US,on="state_name",how="left")
cities_world_stateUSA["state_id"]=cities_world_stateUSA["state_id"].astype(str)
cities_world_stateUSA=cities_world_stateUSA.drop(columns="iso3_y")
cities_world_stateUSA.loc[cities_world_stateUSA['iso3_x'] != "USA", "state_id"] = "nan"
cities_world_stateUSA.columns=["city","country","iso3","iso2","state_name","population","capital","state_id"]
cities_world_stateUSA.head()

,city,country,iso3,iso2,state_name,population,capital,state_id
0,Malishevë,Kosovo,XKS,XK,Malishevë,0.0,admin,nan
1,Prizren,Kosovo,XKS,XK,Prizren,0.0,admin,nan
2,Zubin Potok,Kosovo,XKS,XK,Zubin Potok,0.0,admin,nan
3,Kamenicë,Kosovo,XKS,XK,Kamenicë,0.0,admin,nan
4,Viti,Kosovo,XKS,XK,Viti,0.0,admin,nan


In [27]:
legendaCA=legendaCA[["city","admin","iso2","population"]]
legendaCA["iso2"]="CAN"
legendaCA.columns=["city","state_name","iso3","population"]
legendaCA.head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,city,state_name,iso3,population
0,Toronto,Ontario,CAN,5213000
1,Montréal,Québec,CAN,3678000
2,Vancouver,British Columbia,CAN,2313328
3,Ottawa,Ontario,CAN,1145000
4,Calgary,Alberta,CAN,1110000


In [28]:
s=legendaCA["state_name"].unique()
stati_CA=pd.DataFrame(columns=["state_name","iso3"],data={"state_name":s,"iso3":"CAN"})

In [29]:
legenda_USA_CA=pd.concat([legendaUS,legendaCA],axis=0)
legenda_USA_CA.drop(columns="state_id")
legenda_USA_CA=legenda_USA_CA[["city","state_name","iso3","population"]]
legenda_USA_CA.tail()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,city,state_name,iso3,population
242,Burwash Landing,Yukon,CAN,73.0
243,Grise Fiord,Nunavut,CAN,23.0
244,Big Beaverhouse,Ontario,CAN,10.0
245,Island Lake,Manitoba,CAN,10.0
246,Ennadai,Nunavut,CAN,0.0


In [34]:
gara_tweet["place_user"].head(10).to_frame()

,place_user
0,Tehran
1,United States
2,"Washington, DC"
3,"Noisy-le-Roi, France"
4,"San Diego, CA"
5,"Lake Elsinore, CA"
6,NaN
7,"Edmonton, Alberta Canada"
8,Caracas/Venezuela
9,Senegal


In [18]:
split=gara_tweet["place_user"].str.split(",",expand=True,n=3)
split.head()

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Internet,None,None,None
2,Internet,None,None,None
3,Dallas,TX,None,None
4,NaN,NaN,NaN,NaN


In [151]:
gara_tweet=pd.concat([gara_tweet, split], axis=1)
gara_tweet.columns=["id","place_user","team","info1","info2","info3","info4"]
gara_tweet=gara_tweet[["id","place_user","info1","info2","info3","info4","team"]]
gara_tweet.head()

,id,place_user,info1,info2,info3,info4,team
0,1134248118231416833,Tehran,Tehran,None,None,None,Golden State Warriors
1,1134248122291560448,United States,United States,None,None,None,Golden State Warriors
2,1134248146668851200,"Washington, DC",Washington,DC,None,None,Golden State Warriors
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-le-Roi,France,None,None,Golden State Warriors
4,1134248161931751425,"San Diego, CA",San Diego,CA,None,None,Golden State Warriors


In [152]:
gara_tweet["place_user"]=gara_tweet["place_user"].astype(str)
gara_tweet["place_user"]=gara_tweet["place_user"].fillna("nan")
gara_tweet["place_user"]=gara_tweet["place_user"].replace("None","nan")

gara_tweet["info1"]=gara_tweet["info1"].astype(str)
gara_tweet["info1"]=gara_tweet["info1"].str.strip()
gara_tweet["info1"]=gara_tweet["info1"].str.title()
gara_tweet["info1"]=gara_tweet["info1"].fillna("nan")
gara_tweet["info1"]=gara_tweet["info1"].replace("None","nan")

gara_tweet["info2"]=gara_tweet["info2"].astype(str)
gara_tweet["info2"]=gara_tweet["info2"].str.strip()
gara_tweet["info2"]=gara_tweet["info2"].fillna("nan")
gara_tweet["info2"]=gara_tweet["info2"].replace("None","nan")

gara_tweet["info3"]=gara_tweet["info3"].astype(str)
gara_tweet["info3"]=gara_tweet["info3"].str.strip()
gara_tweet["info3"]=gara_tweet["info3"].fillna("nan")
gara_tweet["info3"]=gara_tweet["info3"].replace("None","nan")

gara_tweet["info4"]=gara_tweet["info4"].astype(str)
gara_tweet["info4"]=gara_tweet["info4"].str.strip()
gara_tweet["info4"]=gara_tweet["info4"].fillna("nan")
gara_tweet["info4"]=gara_tweet["info4"].replace("None","nan")

gara_tweet.head()

,id,place_user,info1,info2,info3,info4,team
0,1134248118231416833,Tehran,Tehran,nan,nan,nan,Golden State Warriors
1,1134248122291560448,United States,United States,nan,nan,nan,Golden State Warriors
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,Golden State Warriors
4,1134248161931751425,"San Diego, CA",San Diego,CA,nan,nan,Golden State Warriors


In [153]:
def match_geoText(citta):
    risultato=GeoText(citta).cities
    if len(risultato)>=1:
        return risultato[0]
    else:
        return 'None'

In [154]:
def update_df(ris):
    global df_infoagg, id_trovati
    ris=ris[["id","city","state_name","iso3","team"]]
    df_infoagg=pd.concat([df_infoagg,ris],axis=0)
    df_infoagg=df_infoagg.reset_index(drop=True)
    df_infoagg=df_infoagg.drop_duplicates()
    id_trovati=df_infoagg["id"].values

In [155]:
def risolviDisambig_CA(df_dasistemare):
    stato="CA"
    citta_CAL=legendaUS[legendaUS["state_id"]==stato]
    estrai=df_dasistemare[df_dasistemare["city"]!="Ontario"]
    risultatoMerge=pd.merge(estrai,citta_CAL,on="city",how="inner")
    return risultatoMerge

In [156]:
def creaLista_info(df):
    lista =[] 
    for r in df.itertuples():
        if (r.country=="nan"):
            ris_join=stati_code[stati_code["iso3"]==r.iso3]
            #print(ris_join)
            info =[str(ris_join["Country"]).upper(), str(r.iso3).upper(), str(ris_join["iso2"]).upper(),str(r.state_name).upper()]
        else:
            info =[str(r.country).upper(), str(r.iso3).upper(), str(r.iso2).upper(),str(r.state_name).upper()]
        info=[str(i) for i in info]
        lista.append(info)  
    return lista

In [157]:
def trova_matchCorretto(place,infoAgg):
    place=place.upper()
    if any(info in place for info in infoAgg):
        return True
    else:
        return False

# Estraggo solo le città scritte correttamente con geotext

In [158]:
risultatoMatch_geo=gara_tweet

In [159]:
risultatoMatch_geo["city"]=risultatoMatch_geo["place_user"].apply(lambda stringa: match_geoText(stringa))
risultatoMatch_geo.head()

,id,place_user,info1,info2,info3,info4,team,city
0,1134248118231416833,Tehran,Tehran,nan,nan,nan,Golden State Warriors,None
1,1134248122291560448,United States,United States,nan,nan,nan,Golden State Warriors,None
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,Golden State Warriors,None
4,1134248161931751425,"San Diego, CA",San Diego,CA,nan,nan,Golden State Warriors,San Diego


In [160]:
risultatoMatch_geo[["place_user","info1","info2","info3","info4","city"]].head()

,place_user,info1,info2,info3,info4,city
0,Tehran,Tehran,nan,nan,nan,None
1,United States,United States,nan,nan,nan,None
2,"Washington, DC",Washington,DC,nan,nan,Washington
3,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,None
4,"San Diego, CA",San Diego,CA,nan,nan,San Diego


In [161]:
risultatoMatch_geo=risultatoMatch_geo[risultatoMatch_geo["city"]!="None"]
len(risultatoMatch_geo)

89692

In [165]:
df_infoagg=pd.DataFrame(columns=["id","city","state_name","iso3","team"])
id_trovati=[]

# Primo join: risultato match geotext con dbUsa (escludendo lo stato CA)

In [164]:
legendaUS_noCA=legendaUS[legendaUS["state_id"]!="CA"]

In [64]:
join_geo_dbUsa=pd.merge(risultatoMatch_geo,legendaUS_noCA,on="city",how="inner")
risultatiUSA_noCA=join_geo_dbUsa[join_geo_dbUsa["info2"]==join_geo_dbUsa["state_id"]]
len(risultatiUSA_noCA)

20841

In [166]:
join_geo_dbUsa.head()

,id,place_user,info1,info2,info3,info4,team,city,state_id,state_name,population,iso3
0,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,VA,Virginia,127.0,USA
1,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,WV,West Virginia,0.0,USA
3,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,NJ,New Jersey,15533.0,USA
4,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,TX,Texas,0.0,USA


In [167]:
join_geo_dbUsa[["place_user","info1","info2","city","state_id","state_name","iso3"]].head()

,place_user,info1,info2,city,state_id,state_name,iso3
0,"Washington, DC",Washington,DC,Washington,VA,Virginia,USA
1,"Washington, DC",Washington,DC,Washington,DC,District of Columbia,USA
2,"Washington, DC",Washington,DC,Washington,WV,West Virginia,USA
3,"Washington, DC",Washington,DC,Washington,NJ,New Jersey,USA
4,"Washington, DC",Washington,DC,Washington,TX,Texas,USA


In [73]:
risultatiUSA_noCA.head()

,id,place_user,info1,info2,info3,info4,team,city,state_id,state_name,population,iso3
1,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA
21,1134250383830896640,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA
141,1134262039336161280,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA
201,1134262960199675904,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA
241,1134263268544004104,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,DC,District of Columbia,5289420.0,USA


In [81]:
update_df(risultatiUSA_noCA)
len(id_trovati)

20841

In [168]:
risultatoMatch_geoMancanti=risultatoMatch_geo[~risultatoMatch_geo["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

89692

In [169]:
risultatoMatch_geoMancanti.head()

,id,place_user,info1,info2,info3,info4,team,city
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington
4,1134248161931751425,"San Diego, CA",San Diego,CA,nan,nan,Golden State Warriors,San Diego
5,1134248169674485760,"Lake Elsinore, CA",Lake Elsinore,CA,nan,nan,Golden State Warriors,Lake Elsinore
7,1134248228449411072,"Edmonton, Alberta Canada",Edmonton,Alberta Canada,nan,nan,Golden State Warriors,Edmonton
8,1134248241132908549,Caracas/Venezuela,Caracas/Venezuela,nan,nan,nan,Golden State Warriors,Caracas


In [84]:
ris_CA=risultatoMatch_geoMancanti[risultatoMatch_geoMancanti["info2"]=="CA"]
len(ris_CA)

5776

# Caso particolare CA

In [50]:
ris=risolviDisambig_CA(ris_CA)
ris=ris[["id","city","state_name","iso3","team"]]
update_df(ris)
len(id_trovati)

37585

In [52]:
ris_Ont=risultatoMatch_geoMancanti[risultatoMatch_geoMancanti["city"]=="Ontario"]
ris_Ont=ris_Ont[["id","team","city"]]
ris_Ont.columns=["id","team","state"]

In [53]:
ris_Ont=pd.merge(ris_Ont,statiCA_capoluogo,on="state",how="inner")

In [54]:
ris_Ont.head()

,id,team,state,city,country
0,1139323129451466753,Golden State Warriors,Ontario,Toronto,CAN
1,1139323487456284672,Golden State Warriors,Ontario,Toronto,CAN
2,1139327091411959813,Golden State Warriors,Ontario,Toronto,CAN
3,1139329825838120960,Golden State Warriors,Ontario,Toronto,CAN
4,1139334924647981057,Golden State Warriors,Ontario,Toronto,CAN


In [55]:
ris_Ont.columns=["id","team","state_name","city","iso3"]
ris_Ont.head()

,id,team,state_name,city,iso3
0,1139323129451466753,Golden State Warriors,Ontario,Toronto,CAN
1,1139323487456284672,Golden State Warriors,Ontario,Toronto,CAN
2,1139327091411959813,Golden State Warriors,Ontario,Toronto,CAN
3,1139329825838120960,Golden State Warriors,Ontario,Toronto,CAN
4,1139334924647981057,Golden State Warriors,Ontario,Toronto,CAN


In [170]:
update_df(ris_Ont)
len(id_trovati)

NameError: name 'ris_Ont' is not defined

In [171]:
risultatoMatch_geoMancanti=risultatoMatch_geo[~risultatoMatch_geo["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

89692

In [172]:
risultatoMatch_geoMancanti.head()

,id,place_user,info1,info2,info3,info4,team,city
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington
4,1134248161931751425,"San Diego, CA",San Diego,CA,nan,nan,Golden State Warriors,San Diego
5,1134248169674485760,"Lake Elsinore, CA",Lake Elsinore,CA,nan,nan,Golden State Warriors,Lake Elsinore
7,1134248228449411072,"Edmonton, Alberta Canada",Edmonton,Alberta Canada,nan,nan,Golden State Warriors,Edmonton
8,1134248241132908549,Caracas/Venezuela,Caracas/Venezuela,nan,nan,nan,Golden State Warriors,Caracas


# Secondo join: risultato match geotext (rimanenti) con dbCanada

In [173]:
ris=pd.merge(risultatoMatch_geoMancanti,legendaCA,on="city",how="inner")
len(ris)

30143

In [174]:
ris[["place_user","info1","info2","city","state_name","iso3"]].head()

,place_user,info1,info2,city,state_name,iso3
0,"Edmonton, Alberta Canada",Edmonton,Alberta Canada,Edmonton,Alberta,CAN
1,"Edmonton, Alberta",Edmonton,Alberta,Edmonton,Alberta,CAN
2,"Edmonton, Alberta",Edmonton,Alberta,Edmonton,Alberta,CAN
3,"Edmonton, Alberta",Edmonton,Alberta,Edmonton,Alberta,CAN
4,"Edmonton, Alberta Canada",Edmonton,Alberta Canada,Edmonton,Alberta,CAN


In [175]:
update_df(ris)
len(id_trovati)

30143

In [176]:
risultatoMatch_geoMancanti=risultatoMatch_geo[~risultatoMatch_geo["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

59738

# Terzo join: risultato match (rimanenti) con dbMondo

In [177]:
ris=pd.merge(risultatoMatch_geoMancanti,cities_world,on="city",how="inner")
lista_info=creaLista_info(ris)
ris["info"]=lista_info
ris["state_name"]=ris["state_name"].astype(str)
ris.head()

,id,place_user,info1,info2,info3,info4,team,city,country,iso3,iso2,state_name,population,capital,info
0,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,"[UNITED STATES, USA, US, DISTRICT OF COLUMBIA]"
1,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,New Jersey,15533.0,NaN,"[UNITED STATES, USA, US, NEW JERSEY]"
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,North Carolina,16138.0,NaN,"[UNITED STATES, USA, US, NORTH CAROLINA]"
3,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Pennsylvania,13566.0,NaN,"[UNITED STATES, USA, US, PENNSYLVANIA]"
4,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Illinois,16811.0,NaN,"[UNITED STATES, USA, US, ILLINOIS]"


In [181]:
ris[["place_user","info1","info2","city","country","state_name","iso3"]].tail()

,place_user,info1,info2,city,country,state_name,iso3
104669,Kuching,Kuching,nan,Kuching,Malaysia,Sarawak,MYS
104670,"Laurinburg, NC",Laurinburg,NC,Laurinburg,United States,North Carolina,USA
104671,Volgograd,Volgograd,nan,Volgograd,Russia,Volgogradskaya Oblast’,RUS
104672,"Sahuarita, AZ",Sahuarita,AZ,Sahuarita,United States,Arizona,USA
104673,"Odense, Danmark",Odense,Danmark,Odense,Denmark,Syddanmark,DNK


In [182]:
prova=ris.apply(lambda x:trova_matchCorretto(x["place_user"],x["info"]),axis=1)
indexMatch=prova[prova.values==True].index
indexMatch=np.array(indexMatch)
len(indexMatch)

19768

In [183]:
risMatch=ris[ris.index.isin(indexMatch)]
risMatch=risMatch[["id","city","state_name","iso3","country","team"]]
risMatch.head()

,id,city,state_name,iso3,country,team
18,1134251019116863488,Washington,District of Columbia,USA,United States,Golden State Warriors
19,1134251019116863488,Washington,New Jersey,USA,United States,Golden State Warriors
20,1134251019116863488,Washington,North Carolina,USA,United States,Golden State Warriors
21,1134251019116863488,Washington,Pennsylvania,USA,United States,Golden State Warriors
22,1134251019116863488,Washington,Illinois,USA,United States,Golden State Warriors


In [184]:
update_df(risMatch)
len(id_trovati)

49905

In [185]:
risultatoMatch_geoMancanti=risultatoMatch_geo[~risultatoMatch_geo["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

44007

In [186]:
def aggiungi_info(lista,info):
    ris_lista=[]
    if info!="nan":
        ris_lista.append(lista)
        ris_lista = [item for lista in ris_lista for item in lista] #per flattare la lista
        ris_lista.append(info)
    else:
        ris_lista=lista
    return ris_lista

In [187]:
def risolvi_disambiguazione(df,id):
    df_result=pd.DataFrame()
    for i in range(len(id)):
        df_match=df[df["id"]==id[i]]
        bestMatch=df_match.loc[df_match["population"].idxmax()].to_frame().T
        df_result=pd.concat([df_result,bestMatch],axis=0)
    return df_result

In [188]:
def EDNormalizzata(stringaDB,stringaCitta,n):
    costo=nltk.edit_distance(stringaDB,stringaCitta)
    return 1-(costo/n)

In [190]:
def ottieni_dfpulitodafunzione(df_risultato,df_apply):
    df=pd.DataFrame()
    for i in range(len(df_risultato)):
        df=pd.concat([df,df_risultato.iloc[i]])   
        df=df.reset_index(drop=True)
    return df

In [191]:
def estraiCitta_nofor(c,i,id_tweet,team):
    i=i.upper()
    estrai=legendaUS[legendaUS["state_id"]==i]
    statoCitta=pd.DataFrame(columns=["id","city","team"],data={'id':id_tweet,'city':c,"team":team},index=[0])
    risultatoMerge=pd.merge(statoCitta,estrai,on="city",how="inner")
    #risultatoMerge=risultatoMerge.drop(columns=["id"])
    return risultatoMerge

In [192]:
def traduci_Citta(c):
    translator = Translator()
    salva=translator.translate(c,dest="en")
    return salva.text

In [193]:
ris=pd.merge(risultatoMatch_geoMancanti,cities_world_stateUSA,on="city",how="inner")
ris.head()

,id,place_user,info1,info2,info3,info4,team,city,country,iso3,iso2,state_name,population,capital,state_id
0,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,DC
1,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,New Jersey,15533.0,NaN,NJ
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,North Carolina,16138.0,NaN,NC
3,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Pennsylvania,13566.0,NaN,PA
4,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Illinois,16811.0,NaN,IL


In [194]:
lista_info=creaLista_info(ris)
ris["info"]=lista_info
lista_info=ris.apply(lambda x: aggiungi_info(x["info"],x["state_id"]),axis=1)
ris["info"]=lista_info
ris.head()

,id,place_user,info1,info2,info3,info4,team,city,country,iso3,iso2,state_name,population,capital,state_id,info
0,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,DC,"[UNITED STATES, USA, US, DISTRICT OF COLUMBIA,..."
1,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,New Jersey,15533.0,NaN,NJ,"[UNITED STATES, USA, US, NEW JERSEY, NJ]"
2,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,North Carolina,16138.0,NaN,NC,"[UNITED STATES, USA, US, NORTH CAROLINA, NC]"
3,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Pennsylvania,13566.0,NaN,PA,"[UNITED STATES, USA, US, PENNSYLVANIA, PA]"
4,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Illinois,16811.0,NaN,IL,"[UNITED STATES, USA, US, ILLINOIS, IL]"


In [195]:
prova=ris.apply(lambda x: trova_matchCorretto(x["place_user"],x["info"]),axis=1)
indexMatch=prova[prova.values==True].index
indexMatch=np.array(indexMatch)
len(indexMatch)

27555

In [196]:
risMatch=ris[ris.index.isin(indexMatch)]
risMatch_count=risMatch.groupby("id")["place_user"].count()
risMatch_count=risMatch_count.to_frame()
risMatch_count=risMatch_count.reset_index()
risMatch_count.sort_values(by="place_user",ascending=False)
risMatch_count.columns=["id","count"]
risMatch.head()

,id,place_user,info1,info2,info3,info4,team,city,country,iso3,iso2,state_name,population,capital,state_id,info
0,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,DC,"[UNITED STATES, USA, US, DISTRICT OF COLUMBIA,..."
5,1134248146668851200,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Indiana,13117.0,NaN,IN,"[UNITED STATES, USA, US, INDIANA, IN]"
9,1134250383830896640,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,DC,"[UNITED STATES, USA, US, DISTRICT OF COLUMBIA,..."
14,1134250383830896640,"Washington, DC",Washington,DC,nan,nan,Golden State Warriors,Washington,United States,USA,US,Indiana,13117.0,NaN,IN,"[UNITED STATES, USA, US, INDIANA, IN]"
18,1134251519568678914,"Washington, DC/Baltimore, MD",Washington,DC/Baltimore,MD,nan,Golden State Warriors,Washington,United States,USA,US,District of Columbia,5289420.0,primary,DC,"[UNITED STATES, USA, US, DISTRICT OF COLUMBIA,..."


In [197]:
id_uniq=risMatch["id"].unique()
ris_dis=risolvi_disambiguazione(risMatch,id_uniq)

In [198]:
update_df(ris_dis)
len(id_trovati)

76255

In [199]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)
risultatoMatch_geoMancanti.head()

,id,place_user,info1,info2,info3,info4,team,city
12,1134248309307060224,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
19,1134248399706869760,"Porac, Central Luzon",Porac,Central Luzon,nan,nan,Golden State Warriors,Porac
37,1134248776099491841,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
42,1134248935382368258,"Hollywood, CA",Hollywood,CA,nan,nan,Golden State Warriors,Hollywood
55,1134249122402226176,"Santa maria,Bulacan📍",Santa Maria,Bulacan📍,nan,nan,Golden State Warriors,Bulacan


In [200]:
ris=pd.merge(risultatoMatch_geoMancanti,cities_world_stateUSA,on="city",how="inner")
ris.head()

,id,place_user,info1,info2,info3,info4,team,city,country,iso3,iso2,state_name,population,capital,state_id
0,1134248935382368258,"Hollywood, CA",Hollywood,CA,nan,nan,Golden State Warriors,Hollywood,United States,USA,US,Florida,153627.0,NaN,FL
1,1134248935382368258,"Hollywood, CA",Hollywood,CA,nan,nan,Golden State Warriors,Hollywood,United States,USA,US,South Carolina,5179.0,NaN,SC
2,1134248429490769921,Hollywood,Hollywood,nan,nan,nan,Toronto Raptors,Hollywood,United States,USA,US,Florida,153627.0,NaN,FL
3,1134248429490769921,Hollywood,Hollywood,nan,nan,nan,Toronto Raptors,Hollywood,United States,USA,US,South Carolina,5179.0,NaN,SC
4,1134263688947523584,Black Political Hollywood (DC),Black Political Hollywood (Dc),nan,nan,nan,Toronto Raptors,Hollywood,United States,USA,US,Florida,153627.0,NaN,FL


In [201]:
id_uniq=ris["id"].unique()
ris_dis=risolvi_disambiguazione(ris,id_uniq)
len(ris_dis)

9081

In [202]:
update_df(ris_dis)
len(id_trovati)

85336

In [203]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

8576

In [204]:
risultatoMatch_geoMancanti.head()

,id,place_user,info1,info2,info3,info4,team,city
12,1134248309307060224,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
19,1134248399706869760,"Porac, Central Luzon",Porac,Central Luzon,nan,nan,Golden State Warriors,Porac
37,1134248776099491841,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
55,1134249122402226176,"Santa maria,Bulacan📍",Santa Maria,Bulacan📍,nan,nan,Golden State Warriors,Bulacan
108,1134250052564602880,"March ARB, California",March Arb,California,nan,nan,Golden State Warriors,March


In [205]:
ris=risultatoMatch_geoMancanti[["place_user","id","info1","info2","info3","info4","team","city"]]

# Join tra città ottenute con geotext (rimanenti) e dbUSA_CANADA

In [206]:
ris_stateUS_CA=pd.merge(ris,legenda_USA_CA,on="city",how="inner")
ris_stateUS_CA.head()

,place_user,id,info1,info2,info3,info4,team,city,state_name,iso3,population
0,Colorado clark,1134250426994372609,Colorado Clark,nan,nan,nan,Golden State Warriors,Colorado,Alaska,USA,0.0
1,"Colorado, USA",1134268751711358976,Colorado,USA,nan,nan,Golden State Warriors,Colorado,Alaska,USA,0.0
2,"Colorado, USA",1134268969811140610,Colorado,USA,nan,nan,Golden State Warriors,Colorado,Alaska,USA,0.0
3,"Colorado, USA",1134269223939633153,Colorado,USA,nan,nan,Golden State Warriors,Colorado,Alaska,USA,0.0
4,"Colorado, USA",1134293954592092160,Colorado,USA,nan,nan,Golden State Warriors,Colorado,Alaska,USA,0.0


In [207]:
ris_stateUS_CA["country"]="nan"
lista_info=creaLista_info(ris_stateUS_CA)
ris_stateUS_CA["info"]=lista_info
#ris["index"]=ris.index
ris_stateUS_CA["state_name"]=ris_stateUS_CA["state_name"].astype(str)
prova=ris_stateUS_CA.apply(lambda x:trova_matchCorretto(x["place_user"],x["info"]),axis=1)
indexMatch=prova[prova.values==True].index
indexMatch=np.array(indexMatch)
len(indexMatch)

693

In [208]:
risMatch=ris_stateUS_CA[ris_stateUS_CA.index.isin(indexMatch)]
id_uniq=risMatch["id"].unique()
ris_dis=risolvi_disambiguazione(risMatch,id_uniq)
len(ris_dis)

493

In [209]:
update_df(ris_dis)
len(id_trovati)

85829

In [210]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)
risultatoMatch_geoMancanti.head()

,id,place_user,info1,info2,info3,info4,team,city
12,1134248309307060224,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
19,1134248399706869760,"Porac, Central Luzon",Porac,Central Luzon,nan,nan,Golden State Warriors,Porac
37,1134248776099491841,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
55,1134249122402226176,"Santa maria,Bulacan📍",Santa Maria,Bulacan📍,nan,nan,Golden State Warriors,Bulacan
108,1134250052564602880,"March ARB, California",March Arb,California,nan,nan,Golden State Warriors,March


# join tra risultato ottenuto cella prima e statiUS_capoluogo

In [211]:
ris_stateUS=pd.merge(risultatoMatch_geoMancanti,statiUS_capoluogo,left_on="info1",right_on="state_name",how="inner")
len(ris_stateUS)

1254

In [212]:
ris_stateUS=ris_stateUS[["id","city_y","state_name","iso3","team"]]
ris_stateUS.columns=["id","city","state_name","iso3","team"]
update_df(ris_stateUS)
len(id_trovati)

87083

In [213]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]

In [215]:
ris_info2=pd.merge(risultatoMatch_geoMancanti,stati_US,left_on="info2",right_on="state_name")
len(ris_info2)

239

In [216]:
ris_info2.head()

,id,place_user,info1,info2,info3,info4,team,city,state_id,state_name,iso3
0,1134250052564602880,"March ARB, California",March Arb,California,nan,nan,Golden State Warriors,March,CA,California,USA
1,1134266487894528001,"March ARB, California",March Arb,California,nan,nan,Golden State Warriors,March,CA,California,USA
2,1134261573122269184,"March ARB, California",March Arb,California,nan,nan,Toronto Raptors,March,CA,California,USA
3,1134264154494197761,"March ARB, California",March Arb,California,nan,nan,Toronto Raptors,March,CA,California,USA
4,1134293970333315072,"March ARB, California",March Arb,California,nan,nan,Toronto Raptors,March,CA,California,USA


In [217]:
ris_Simil=ris_info2.apply(lambda x: estraiCitta_Simil_nofor(x['info1'],x['state_id'],x["id"],x["team"]),axis=1)
len(ris_Simil)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


239

# Definisco soglia 0.3 per jaccard distance (se <= match altrimenti no)

In [261]:
ris=ottieni_dfpulitodafunzione(ris_Simil,ris_info2)
risultatiMigliori=ris[ris["best_jacc"]==ris["best_edit"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["val_jacc"]<=0.3]
risultatiMigliori["city"]=risultatiMigliori["best_jacc"]
risultatiMigliori.head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


KeyboardInterrupt: 

In [219]:
ris=pd.merge(risultatiMigliori,legendaUS,on="city",how="inner")
update_df(ris)
len(id_trovati)

87392

In [220]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

6717

In [221]:
rimanenti=risultatoMatch_geoMancanti
rimanenti.head()

,id,place_user,info1,info2,info3,info4,team,city
12,1134248309307060224,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
19,1134248399706869760,"Porac, Central Luzon",Porac,Central Luzon,nan,nan,Golden State Warriors,Porac
37,1134248776099491841,Mariel's,Mariel'S,nan,nan,nan,Golden State Warriors,Mariel
55,1134249122402226176,"Santa maria,Bulacan📍",Santa Maria,Bulacan📍,nan,nan,Golden State Warriors,Bulacan
108,1134250052564602880,"March ARB, California",March Arb,California,nan,nan,Golden State Warriors,March


In [222]:
ris=pd.merge(rimanenti,statiUS_capoluogo,left_on="info1",right_on="state_name",how="inner")
ris=ris[["id","place_user","state_name","iso3","city_y","team"]]
ris.columns=["id","place_user","state_name","iso3","city","team"]

,id,place_user,state_name,iso3,city,team


In [223]:
update_df(ris)
len(id_trovati)

87392

In [224]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

6717

In [225]:
risultatoMatch_geoMancanti.head()
ris=pd.merge(risultatoMatch_geoMancanti,statiUS_capoluogo,left_on='city', right_on='state_name',how="inner")
ris=ris[["id","city_y","state_name","iso3","team"]]
ris.columns=["id","city","state_name","iso3","team"]
len(ris)

278

In [226]:
update_df(ris)
len(id_trovati)

87670

In [227]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

6439

In [228]:
ris=pd.merge(risultatoMatch_geoMancanti,legenda_USA_CA,on="city",how="inner")
len(ris)

5094

In [229]:
update_df(ris)
len(id_trovati)

92764

In [230]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

3827

In [231]:
ris=pd.merge(risultatoMatch_geoMancanti,stati_code,left_on="info2",right_on="Country")
ris["state_name"]="nan"
ris=ris[["id","city","state_name","iso3","Country","team"]]
len(ris)

312

In [232]:
update_df(ris)
len(id_trovati)

93076

In [233]:
risultatoMatch_geoMancanti=risultatoMatch_geoMancanti[~risultatoMatch_geoMancanti["id"].isin(id_trovati)]
len(risultatoMatch_geoMancanti)

3515

In [234]:
id_geoNontrovati=risultatoMatch_geoMancanti["id"].values

In [236]:
ris=pd.merge(risultatoMatch_geoMancanti,stati_CA,left_on="info2",right_on="state_name")
update_df(ris)
len(id_trovati)

94220

In [237]:
pos_noCity=gara_tweet[~gara_tweet["id"].isin(id_trovati)]
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_geoNontrovati)]
len(pos_noCity)

136409

In [238]:
pos_noCity.tail()

,id,place_user,info1,info2,info3,info4,team,city
226094,1134320168732020738,Central City,Central City,nan,nan,nan,Neutro,None
226095,1134320170099286017,Anfield Stadium,Anfield Stadium,nan,nan,nan,Neutro,None
226096,1134320171995189249,Beyond Reality,Beyond Reality,nan,nan,nan,Neutro,None
226097,1134320172792045569,Ib,Ib,nan,nan,nan,Neutro,None
226100,1134320177296752640,send location,Send Location,nan,nan,nan,Neutro,None


In [239]:
pos_noCity.head()

,id,place_user,info1,info2,info3,info4,team,city
0,1134248118231416833,Tehran,Tehran,nan,nan,nan,Golden State Warriors,None
1,1134248122291560448,United States,United States,nan,nan,nan,Golden State Warriors,None
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,Golden State Warriors,None
6,1134248216604467200,nan,Nan,nan,nan,nan,Golden State Warriors,None
9,1134248269855502336,Senegal,Senegal,nan,nan,nan,Golden State Warriors,None


# Join tra tweet in cui GeoText non è riuscito ad estrarre città con db capitaliWorld (join con country)

In [240]:
ris=pd.merge(pos_noCity,capitali_world,left_on="info1",right_on="country",how="inner")
ris=ris[ris["info2"]=="nan"]
ris.head()

,id,place_user,info1,info2,info3,info4,team,city_x,city_y,country,iso3,iso2,state_name,population,capital
0,1134248122291560448,United States,United States,nan,nan,nan,Golden State Warriors,None,Washington,United States,USA,US,District of Columbia,5289420.0,primary
1,1134248695132594176,United States,United States,nan,nan,nan,Golden State Warriors,None,Washington,United States,USA,US,District of Columbia,5289420.0,primary
2,1134249836587429888,United States,United States,nan,nan,nan,Golden State Warriors,None,Washington,United States,USA,US,District of Columbia,5289420.0,primary
3,1134252400565469184,United States,United States,nan,nan,nan,Golden State Warriors,None,Washington,United States,USA,US,District of Columbia,5289420.0,primary
4,1134253624383672325,United States,United States,nan,nan,nan,Golden State Warriors,None,Washington,United States,USA,US,District of Columbia,5289420.0,primary


In [129]:
ris=ris[["id","team","place_user","city_y","iso3","iso2","state_name","country"]]
ris.columns=["id","team","place_user","city","iso3","iso2","state_name","country"]
len(ris)

17977

In [130]:
lista_info=creaLista_info(ris)
ris["info"]=lista_info
#ris["index"]=ris.index
ris["state_name"]=ris["state_name"].astype(str)

prova=ris.apply(lambda x:trova_matchCorretto(x["place_user"],x["info"]),axis=1)
indexMatch=prova[prova.values==True].index
indexMatch=np.array(indexMatch)
len(indexMatch)

17977

In [131]:
update_df(ris)
len(id_trovati)

160954

In [132]:
pos_noCity=gara_tweet[~gara_tweet["id"].isin(id_trovati)]
pos_noCity.head()

,id,place_user,info1,info2,info3,info4,team,city
0,1139321556264464390,nan,Nan,nan,nan,nan,Golden State Warriors,None
1,1139321558860591105,Internet,Internet,nan,nan,nan,Golden State Warriors,None
2,1139321563344281600,Internet,Internet,nan,nan,nan,Golden State Warriors,None
4,1139321571531808768,nan,Nan,nan,nan,nan,Golden State Warriors,None
9,1139321644546187265,nan,Nan,nan,nan,nan,Golden State Warriors,None


In [133]:
pos_noCity_USA=pos_noCity[pos_noCity["info2"]=="USA"]
ris=pd.merge(pos_noCity_USA,statiUS_capoluogo,left_on="info1",right_on="state_name",how="inner")
ris=ris[["id","team","place_user","city_y","iso3","state_name"]]
ris.columns=["id","team","place_user","city","iso3","state_name"]

In [134]:
update_df(ris)
len(id_trovati)

166711

# join tra info1 e city

In [135]:
ris=pd.merge(pos_noCity,capitali_world,left_on="info1",right_on="city",how="inner")
len(ris)

407

In [136]:
lista_info=creaLista_info(ris)
ris["info"]=lista_info
#ris["index"]=ris.index
ris["state_name"]=ris["state_name"].astype(str)

prova=ris.apply(lambda x:trova_matchCorretto(x["place_user"],x["info"]),axis=1)
indexMatch=prova[prova.values==True].index
indexMatch=np.array(indexMatch)
len(indexMatch)

193

In [137]:
risMatch=ris[ris.index.isin(indexMatch)]
risMatch=risMatch[["id","city_y","state_name","iso3","country","team"]]
risMatch.columns=["id","city","state_name","iso3","country","team"]

In [138]:
update_df(risMatch)
len(id_trovati)

166904

In [139]:
pos_noCity=gara_tweet[~gara_tweet["id"].isin(id_trovati)]
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_geoNontrovati)]

pos_noCity["info1"]=pos_noCity["info1"].replace("Nan","nan")
pos_noCity["info1"]=pos_noCity["info1"].replace("None","nan")
pos_noCity["info2"]=pos_noCity["info2"].replace("None","nan")
pos_noCity["info3"]=pos_noCity["info3"].replace("None","nan")
pos_noCity["info4"]=pos_noCity["info4"].replace("None","nan")

In [287]:
pos_noCity.head()

,id,place_user,info1,info2,info3,info4,team,city,num_car_info2
0,1134248118231416833,Tehran,Tehran,nan,nan,nan,Golden State Warriors,None,3
1,1134248122291560448,United States,United States,nan,nan,nan,Golden State Warriors,None,3
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,Golden State Warriors,None,6
6,1134248216604467200,nan,nan,nan,nan,nan,Golden State Warriors,None,3
9,1134248269855502336,Senegal,Senegal,nan,nan,nan,Golden State Warriors,None,3


In [289]:
pos_noCity[["place_user","info1","info2","info3","info4"]].head()

,place_user,info1,info2,info3,info4
0,Tehran,Tehran,nan,nan,nan
1,United States,United States,nan,nan,nan
3,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan
6,nan,nan,nan,nan,nan
9,Senegal,Senegal,nan,nan,nan


# estraggo solo i tweet dove info2 contiene valore ed ha lungezza 2

In [242]:
salva=pos_noCity[(pos_noCity["info2"]!="nan")]
salva["num_car_info2"]=salva["info2"].apply(lambda info:len(info))
salva=salva[salva["num_car_info2"]==2]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [262]:
salva.head()

,id,place_user,info1,info2,info3,info4,team,city,num_car_info2
40,1134248908845211654,"Uhrichsville, OH",Uhrichsville,OH,nan,nan,Golden State Warriors,None,2
73,1134249452007387138,"South San Francisco, CA",South San Francisco,CA,nan,nan,Golden State Warriors,None,2
199,1134251650632208384,"San Martin, CA",San Martin,CA,nan,nan,Golden State Warriors,None,2
219,1134251895936114689,"san jose, ca",San Jose,ca,nan,nan,Golden State Warriors,None,2
259,1134252542039171072,"San Martin, CA",San Martin,CA,nan,nan,Golden State Warriors,None,2


In [247]:
ris=salva.apply(lambda x:estraiCitta_nofor(x["info1"],x["info2"],x["id"],x["team"]),axis=1)
ris=ottieni_dfpulitodafunzione(ris,salva)
len(ris)

2313

In [248]:
update_df(ris)
len(id_trovati)

96533

In [249]:
pos_noCity=gara_tweet[~gara_tweet["id"].isin(id_trovati)]
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_geoNontrovati)]
len(pos_noCity)

134096

# Mi concentro sui tweet di cui non si è riusciti ad estrarre la città

In [250]:
pos_noCity.head()
pos_noCity["info1"]=pos_noCity["info1"].replace("Nan","nan")
pos_noCity["info1"]=pos_noCity["info1"].replace("None","nan")

pos_noCity["info2"]=pos_noCity["info2"].replace("None","nan")
#pos_noCity["info2"]=pos_noCity["info2"].replace("NaN","nan")
pos_noCity["info2"]=pos_noCity["info2"].fillna("nan")

pos_noCity["info3"]=pos_noCity["info3"].replace("None","nan")
#pos_noCity["info3"]=pos_noCity["info3"].replace("NaN","nan")
pos_noCity["info3"]=pos_noCity["info3"].fillna("nan")

pos_noCity["info4"]=pos_noCity["info4"].replace("None","nan")
#pos_noCity["info4"]=pos_noCity["info4"].replace("NaN","nan")
pos_noCity["info4"]=pos_noCity["info4"].fillna("nan")

In [147]:
ris=pd.merge(pos_noCity,statiUS_capoluogo,left_on="info1",right_on="state_name")
ris.head()

,id,place_user,info1,info2,info3,info4,team,city_x,city_y,state_id,state_name,lat,lng,population,iso3
0,1139322417262628864,California,California,nan,nan,nan,Golden State Warriors,None,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA
1,1139322760281243648,California,California,nan,nan,nan,Golden State Warriors,None,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA
2,1139322823279726592,California,California,nan,nan,nan,Golden State Warriors,None,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA
3,1139323751982456832,california,California,nan,nan,nan,Golden State Warriors,None,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA
4,1139324533536505857,california,California,nan,nan,nan,Golden State Warriors,None,Los Angeles,CA,California,34.1139,-118.4068,12815475.0,USA


In [148]:
ris=ris[["id","city_y","state_name","iso3","team"]]
ris.columns=["id","city","state_name","iso3","team"]
len(ris)

1988

In [149]:
update_df(ris)
len(id_trovati)

172142

In [251]:
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_trovati)]
pos_noCity.head()
#pos_noCity[pos_noCity["info2"]!="nan"].head()
pos_noCity_Simil=pos_noCity

In [189]:
def estraiCitta_Simil_nofor(c,i,id_tweet,team):
    ris=pd.DataFrame()
    i=i.upper()
    estrai=legendaUS[legendaUS["state_id"]==i]
    risultatoDistance=estrai[["city"]]
    statoCitta=pd.DataFrame(columns=["id","city","team","Country"],data={'id':id_tweet,'city':c,'Country':i,"team":team},index=[0])
    if (len(risultatoDistance)==0):
        i=i.title()
        statoCitta["Country"]=i
        #print(statoCitta)
        ris=pd.merge(statoCitta,stati_code,on="Country",how="inner")
    else:
        if c=="nan":
            ris=pd.merge(statiUS_capoluogo,statoCitta,left_on="state_id",right_on="Country",how="inner")
            print(ris)
        else:
            risultatoDistance["d_jacc"]=risultatoDistance["city"].apply(lambda stringa: nltk.jaccard_distance(set(stringa), set(c)))
            lunghezza_maxCitta=estrai["city"].apply(lambda stringa: len(stringa)).max()
            risultatoDistance["d_editNorm"]=risultatoDistance["city"].apply(lambda stringa1: EDNormalizzata(stringa1,c,lunghezza_maxCitta))
            best_jacc=risultatoDistance.loc[risultatoDistance.d_jacc.idxmin()]
            best_deditNorm= risultatoDistance.loc[risultatoDistance.d_editNorm.idxmax()]
            best_jacc=best_jacc.to_frame().T.reset_index()
            best_jacc=best_jacc.drop(columns=["index","d_editNorm"])
            best_jacc.columns=["best_jacc","val_jacc"]
            best_deditNorm=best_deditNorm.to_frame().T.reset_index()
            best_deditNorm=best_deditNorm.drop(columns=["index","d_jacc"])
            best_deditNorm.columns=["best_edit","val_edit"]
            risultatiMisure=pd.concat([best_jacc,best_deditNorm],axis=1)
            risultatiMisure["city_originale"]=c
            risultatiMisure["id"]=id_tweet
            risultatiMisure["team"]=team
            ris=risultatiMisure
    return ris

In [259]:
pos_noCity_Simil["num_car_info2"]=pos_noCity_Simil["info2"].apply(lambda stringa: len(stringa))
pos_noCity_Simil=pos_noCity_Simil[pos_noCity_Simil["info2"]!="nan"]
pos_noCity_Simil["info1"]=pos_noCity_Simil["info1"].fillna("nan")
pos_noCity_Simil.head()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,place_user,info1,info2,info3,info4,team,city,num_car_info2
3,1134248155149754369,"Noisy-le-Roi, France",Noisy-Le-Roi,France,nan,nan,Golden State Warriors,None,6
63,1134249283648131072,"Illinois, USA",Illinois,USA,nan,nan,Golden State Warriors,None,3
67,1134249361913679872,"California, USA",California,USA,nan,nan,Golden State Warriors,None,3
92,1134249770174697472,"Valenzuela City, National Capi",Valenzuela City,National Capi,nan,nan,Golden State Warriors,None,13
110,1134250061854941184,"California, USA",California,USA,nan,nan,Golden State Warriors,None,3


In [260]:
ris_Simil=pos_noCity_Simil.apply(lambda x: estraiCitta_Simil_nofor(x['info1'],x['info2'],x["id"],x["team"]),axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [263]:
ris=ottieni_dfpulitodafunzione(ris_Simil,pos_noCity_Simil)
len(ris)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


3861

In [290]:
risultatiMigliori=ris[ris["city_originale"]!="nan"]
risultatiMigliori=risultatiMigliori[["best_edit","best_jacc","city_originale","id","team","val_edit","val_jacc"]]
ris1=risultatiMigliori
risultatiMigliori=risultatiMigliori[risultatiMigliori["best_jacc"]==risultatiMigliori["best_edit"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["val_jacc"]<=0.3]
risultatiMigliori["city"]=risultatiMigliori["best_jacc"]
risultatiMigliori.head(10)

,best_edit,best_jacc,city_originale,id,team,val_edit,val_jacc,city
7,East Oakdale,East Oakdale,East Oakland,1134257144264503297,Golden State Warriors,0.88,0.181818,East Oakdale
24,Williston Park,Williston Park,Clifton Park,1134262600609538048,Golden State Warriors,0.818182,0.285714,Williston Park
30,Saint Louis,Saint Louis,St Louis,1134263085919813633,Golden State Warriors,0.869565,0.2,Saint Louis
32,Saint Louis,Saint Louis,St Louis,1134263190005698560,Golden State Warriors,0.869565,0.2,Saint Louis
35,Coal City,Coal City,Choppa City,1134263446327967744,Golden State Warriors,0.809524,0.3,Coal City
64,Saint Louis,Saint Louis,St Louis,1134265612811821061,Golden State Warriors,0.869565,0.2,Saint Louis
89,Polk City,Polk City,Polk County,1134266920801308672,Golden State Warriors,0.888889,0.272727,Polk City
106,Saint Petersburg,Saint Petersburg,St Petersburg,1134268358713630720,Golden State Warriors,0.888889,0.230769,Saint Petersburg
107,Saint Paul,Saint Paul,St Paul,1134268617317597184,Golden State Warriors,0.863636,0.222222,Saint Paul
109,Saint Paul,Saint Paul,St Paul,1134268743352295424,Golden State Warriors,0.863636,0.222222,Saint Paul


In [292]:
risultatiMigliori[["best_edit","best_jacc","city_originale","val_edit","val_jacc","city"]].head()

,best_edit,best_jacc,city_originale,val_edit,val_jacc,city
7,East Oakdale,East Oakdale,East Oakland,0.88,0.181818,East Oakdale
24,Williston Park,Williston Park,Clifton Park,0.818182,0.285714,Williston Park
30,Saint Louis,Saint Louis,St Louis,0.869565,0.2,Saint Louis
32,Saint Louis,Saint Louis,St Louis,0.869565,0.2,Saint Louis
35,Coal City,Coal City,Choppa City,0.809524,0.3,Coal City


In [272]:
ris1.head()

,best_edit,best_jacc,city_originale,id,team,val_edit,val_jacc
0,NaN,NaN,NaN,1134248155149754369,Golden State Warriors,NaN,NaN
1,Brea,Bear Valley,Bay Area,1134252649249820672,Golden State Warriors,0.84,0.333333
2,NaN,NaN,NaN,1134252652160737280,Golden State Warriors,NaN,NaN
3,Brea,Bear Valley,Bay Area,1134253673150685184,Golden State Warriors,0.84,0.333333
4,Thermal,Terra Bella,The Bay,1134254478591324160,Golden State Warriors,0.88,0.444444


In [155]:
ris.head()

,Country,best_edit,best_jacc,city,city_originale,id,iso2,iso3,team,val_edit,val_jacc
0,NaN,Mill Valley,Simi Valley,NaN,Silly Valley,1139321835743375360,NaN,NaN,Golden State Warriors,0.92,0.111111
1,United Kingdom,NaN,NaN,Scotland,NaN,1139322022754803712,GB,GBR,Golden State Warriors,NaN,NaN
2,Philippines,NaN,NaN,Metro Manila,NaN,1139322327982718977,PH,PHL,Golden State Warriors,NaN,NaN
3,Serbia,NaN,NaN,Republic Of Slovenia,NaN,1139322500293091328,RS,SRB,Golden State Warriors,NaN,NaN
4,NaN,Brea,Bear Valley,NaN,Bay Area,1139323279422787584,NaN,NaN,Golden State Warriors,0.84,0.333333


In [156]:
ris["iso3"]=ris["iso3"].fillna("nan")
ris["city_originale"]=ris["city_originale"].fillna("nan")
ris_iso3=ris[ris["iso3"]!="nan"]
len(ris_iso3)

4494

# escludo i tweet dove  field city = "England"

In [157]:
ris_iso3_noEngland=ris_iso3[ris_iso3["city"]!="England"]
len(ris_iso3_noEngland)
ris_iso3_noEngland.head()

,Country,best_edit,best_jacc,city,city_originale,id,iso2,iso3,team,val_edit,val_jacc
1,United Kingdom,NaN,NaN,Scotland,nan,1139322022754803712,GB,GBR,Golden State Warriors,NaN,NaN
2,Philippines,NaN,NaN,Metro Manila,nan,1139322327982718977,PH,PHL,Golden State Warriors,NaN,NaN
3,Serbia,NaN,NaN,Republic Of Slovenia,nan,1139322500293091328,RS,SRB,Golden State Warriors,NaN,NaN
5,Philippines,NaN,NaN,Caloocan City,nan,1139323795502555137,PH,PHL,Golden State Warriors,NaN,NaN
11,Portugal,NaN,NaN,Amarante,nan,1139324646933827584,PT,PRT,Golden State Warriors,NaN,NaN


In [158]:
ris_iso3_noEngland_noUSACAN=ris_iso3_noEngland[(ris_iso3_noEngland["iso3"]!="USA") & (ris_iso3_noEngland["iso3"]!="CAN")]
ris_iso3_noEngland_noUSACAN.head()

,Country,best_edit,best_jacc,city,city_originale,id,iso2,iso3,team,val_edit,val_jacc
1,United Kingdom,NaN,NaN,Scotland,nan,1139322022754803712,GB,GBR,Golden State Warriors,NaN,NaN
2,Philippines,NaN,NaN,Metro Manila,nan,1139322327982718977,PH,PHL,Golden State Warriors,NaN,NaN
3,Serbia,NaN,NaN,Republic Of Slovenia,nan,1139322500293091328,RS,SRB,Golden State Warriors,NaN,NaN
5,Philippines,NaN,NaN,Caloocan City,nan,1139323795502555137,PH,PHL,Golden State Warriors,NaN,NaN
11,Portugal,NaN,NaN,Amarante,nan,1139324646933827584,PT,PRT,Golden State Warriors,NaN,NaN


In [159]:
cities_world_noUSA=cities_world[cities_world["iso3"]!="USA"]

In [160]:
ris_iso=pd.merge(ris_iso3_noEngland_noUSACAN,cities_world_noUSA,on="city",how="inner")
len(ris_iso)

172

In [161]:
ris_iso=ris_iso[ris_iso["iso3_x"]==ris_iso["iso3_y"]]
ris_iso=ris_iso[["id","city","state_name","team","iso3_x"]]
ris_iso.columns= ["id","city","state_name","team","iso3"]

In [162]:
update_df(ris_iso)
len(id_trovati)

172291

In [163]:
ris_iso3_England=ris_iso3[ris_iso3["city"]=="England"]
ris_iso3_England=ris_iso3_England[["Country","iso3","id","team"]]

In [164]:
ris_iso3_England=pd.merge(ris_iso3_England,capitali_world,on="iso3",how="inner")
update_df(ris_iso3_England)
len(id_trovati)

172436

In [165]:
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_trovati)]
pos_noCity.head()

,id,place_user,info1,info2,info3,info4,team,city,num_car_info2
0,1139321556264464390,nan,nan,nan,nan,nan,Golden State Warriors,None,3
1,1139321558860591105,Internet,Internet,nan,nan,nan,Golden State Warriors,None,3
2,1139321563344281600,Internet,Internet,nan,nan,nan,Golden State Warriors,None,3
4,1139321571531808768,nan,nan,nan,nan,nan,Golden State Warriors,None,3
9,1139321644546187265,nan,nan,nan,nan,nan,Golden State Warriors,None,3


In [166]:
pos_noCity["num_car_info2"]=pos_noCity["info2"].apply(lambda stringa:len(stringa))
pos_noCity=pos_noCity[pos_noCity["num_car_info2"]==2]

In [167]:
ris_Simil=pos_noCity.apply(lambda x: estraiCitta_Simil_nofor(x['info1'],x['info2'],x["id"],x["team"]),axis=1)
ris=ottieni_dfpulitodafunzione(ris_Simil,pos_noCity)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.

In [275]:
risultatiMigliori=ris[ris["city_originale"]!="nan"]

In [276]:
risultatiMigliori=risultatiMigliori[["best_edit","best_jacc","city_originale","id","team","val_edit","val_jacc"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["best_jacc"]==risultatiMigliori["best_edit"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["val_jacc"]<=0.3]
risultatiMigliori["city"]=risultatiMigliori["best_jacc"]
risultatiMigliori.head()

,best_edit,best_jacc,city_originale,id,team,val_edit,val_jacc,city
7,East Oakdale,East Oakdale,East Oakland,1134257144264503297,Golden State Warriors,0.88,0.181818,East Oakdale
24,Williston Park,Williston Park,Clifton Park,1134262600609538048,Golden State Warriors,0.818182,0.285714,Williston Park
30,Saint Louis,Saint Louis,St Louis,1134263085919813633,Golden State Warriors,0.869565,0.2,Saint Louis
32,Saint Louis,Saint Louis,St Louis,1134263190005698560,Golden State Warriors,0.869565,0.2,Saint Louis
35,Coal City,Coal City,Choppa City,1134263446327967744,Golden State Warriors,0.809524,0.3,Coal City


In [278]:
risultatiMigliori[["best_edit","best_jacc","city_originale","val_jacc","city"]].head()

,best_edit,best_jacc,city_originale,val_jacc,city
7,East Oakdale,East Oakdale,East Oakland,0.181818,East Oakdale
24,Williston Park,Williston Park,Clifton Park,0.285714,Williston Park
30,Saint Louis,Saint Louis,St Louis,0.2,Saint Louis
32,Saint Louis,Saint Louis,St Louis,0.2,Saint Louis
35,Coal City,Coal City,Choppa City,0.3,Coal City


In [169]:
ris=pd.merge(risultatiMigliori,legendaUS,on="city",how="inner")
update_df(ris)
len(id_trovati)

174460

In [170]:
pos_noCity=pos_noCity[~pos_noCity["id"].isin(id_trovati)]
len(pos_noCity)

2729

# Questo è il numero totale dei match rispetto al dbiniziale

In [171]:
len(gara_tweet[gara_tweet["id"].isin(id_trovati)])

166079

In [172]:
pos_noCityAll=gara_tweet[~gara_tweet["id"].isin(id_trovati)]

# numero tweet che non hanno nessun valore in place_user

In [173]:
len(gara_tweet[gara_tweet["place_user"]=="nan"])

92265

In [174]:
pos_noCityAll_noNan=pos_noCityAll[pos_noCityAll["place_user"]!="nan"]
len(pos_noCityAll_noNan)

103689

In [175]:
ris=pd.merge(pos_noCityAll_noNan,cities_world,left_on="info1",right_on="city",how="inner")
len(ris)
risMatch=ris
ris.head()

,id,place_user,info1,info2,info3,info4,team,city_x,city_y,country,iso3,iso2,state_name,population,capital
0,1139322567783677953,"ILO, PH",Ilo,PH,nan,nan,Golden State Warriors,None,Ilo,Peru,PER,PE,Moquegua,53476.0,NaN
1,1139323640032321536,ilo,Ilo,nan,nan,nan,Golden State Warriors,None,Ilo,Peru,PER,PE,Moquegua,53476.0,NaN
2,1139377872026525696,ilo,Ilo,nan,nan,nan,Golden State Warriors,None,Ilo,Peru,PER,PE,Moquegua,53476.0,NaN
3,1139388885635518464,ilo,Ilo,nan,nan,nan,Toronto Raptors,None,Ilo,Peru,PER,PE,Moquegua,53476.0,NaN
4,1139386700084305921,ILO,Ilo,nan,nan,nan,Neutro,None,Ilo,Peru,PER,PE,Moquegua,53476.0,NaN


In [176]:
risMatch_count=risMatch.groupby("id")["place_user"].count()
risMatch_count=risMatch_count.to_frame()
risMatch_count=risMatch_count.reset_index()
risMatch_count.sort_values(by="place_user",ascending=False)
risMatch_count.columns=["id","count"]
risMatch_count.sort_values(by="count",ascending=False).head()

,id,count
1590,1139378211563827203,9
167,1139335751567433728,8
555,1139348680316612608,8
2149,1139382583932833793,8
2155,1139382652996243456,8


# Id tweet match città

In [177]:
id_tweet=risMatch_count["id"].unique()
id_tweet

array([1139321604780023808, 1139321690461286406, 1139321696530436096, ...,
       1139393995547656193, 1139394001327411200, 1139394021955010560],
      dtype=int64)

In [178]:
ris_dis=risolvi_disambiguazione(ris,id_tweet)
len(ris_dis)

3671

In [179]:
ris_dis.head()
ris_dis=ris_dis[["id","city_y","state_name","iso3","team"]]
ris_dis.columns=["id","city","state_name","iso3","team"]

In [180]:
ris_dis.head()

,id,city,state_name,iso3,team
1019,1139321604780023808,Toronto,Ontario,CAN,Toronto Raptors
1728,1139321690461286406,Toronto,Ontario,CAN,Neutro
2353,1139321696530436096,Swift Current,Saskatchewan,CAN,Toronto Raptors
2369,1139321783423778816,East Orange,New Jersey,USA,Toronto Raptors
15,1139321835349090304,Salt Lake City,Utah,USA,Toronto Raptors


In [181]:
update_df(ris_dis)
len(id_trovati)

178131

In [182]:
pos_noCityAll_noNan=pos_noCityAll_noNan[~pos_noCityAll_noNan["id"].isin(id_trovati)]
len(pos_noCityAll_noNan)

100018

In [183]:
pos_noCityAll_noNan.head()

,id,place_user,info1,info2,info3,info4,team,city
1,1139321558860591105,Internet,Internet,nan,nan,nan,Golden State Warriors,None
2,1139321563344281600,Internet,Internet,nan,nan,nan,Golden State Warriors,None
11,1139321659163201536,"Tipoca City, Kamino",Tipoca City,Kamino,nan,nan,Golden State Warriors,None
16,1139321690050244609,"ÜT: 41.886177,-87.601774",Üt: 41.886177,-87.601774,nan,nan,Golden State Warriors,None
31,1139321822220963841,"Banga, Western Visayas",Banga,Western Visayas,nan,nan,Golden State Warriors,Banga


In [184]:
pos_noCityAll_noNan=pos_noCityAll_noNan[pos_noCityAll_noNan["info2"]!="nan"]
pos_noCityAll_noNan.head(30)

,id,place_user,info1,info2,info3,info4,team,city
11,1139321659163201536,"Tipoca City, Kamino",Tipoca City,Kamino,nan,nan,Golden State Warriors,None
16,1139321690050244609,"ÜT: 41.886177,-87.601774",Üt: 41.886177,-87.601774,nan,nan,Golden State Warriors,None
31,1139321822220963841,"Banga, Western Visayas",Banga,Western Visayas,nan,nan,Golden State Warriors,Banga
32,1139321835743375360,"Silly Valley, CA",Silly Valley,CA,nan,nan,Golden State Warriors,None
38,1139321865975943168,"Bay Area, Califorina",Bay Area,Califorina,nan,nan,Golden State Warriors,None
43,1139321917150658560,"Valenzuela City, National Capi",Valenzuela City,National Capi,nan,nan,Golden State Warriors,None
59,1139322022754803712,"Scotland, United Kingdom",Scotland,United Kingdom,nan,nan,Golden State Warriors,None
72,1139322136516935680,"Parts, Unknown",Parts,Unknown,nan,nan,Golden State Warriors,None
83,1139322327982718977,"metro manila, philippines",Metro Manila,philippines,nan,nan,Golden State Warriors,None
100,1139322500293091328,"Republic of Slovenia , Serbia",Republic Of Slovenia,Serbia,nan,nan,Golden State Warriors,None


In [185]:
ris=pd.merge(pos_noCityAll_noNan,stati_CA,left_on="info2",right_on="state_name",how="inner")
ris=ris[["info1","id","team","state_name","iso3"]]
ris.columns=["city","id","team","state_name","iso3"]

In [186]:
update_df(ris)
len(id_trovati)

178678

In [187]:
pos_noCityAll_noNan=pos_noCityAll_noNan[~pos_noCityAll_noNan["id"].isin(id_trovati)]
len(pos_noCityAll_noNan)

18127

In [188]:
ris=pd.merge(pos_noCityAll_noNan,stati_US,left_on="info2",right_on="state_name",how="inner")
len(ris)

389

In [189]:
ris=pos_noCityAll_noNan.apply(lambda x: estraiCitta_Simil_nofor(x['info1'],x['info2'],x["id"],x["team"]),axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [190]:
ris=ottieni_dfpulitodafunzione(ris,pos_noCityAll_noNan)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  after removing the cwd from sys.path.


In [191]:
len(ris)

5848

In [192]:
risultatiMigliori=ris[ris["city_originale"]!="nan"]
risultatiMigliori=risultatiMigliori[["best_edit","best_jacc","city_originale","id","team","val_edit","val_jacc"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["best_jacc"]==risultatiMigliori["best_edit"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["val_jacc"]<=0.3]
risultatiMigliori["city"]=risultatiMigliori["best_jacc"]
risultatiMigliori.head()

,best_edit,best_jacc,city_originale,id,team,val_edit,val_jacc,city
1143,McKinney,McKinney,Mckinney,1139367912215629824,Toronto Raptors,0.967742,0.25,McKinney
4274,Coconut Creek,Coconut Creek,Coconut Grove,1139365192616144896,Neutro,0.851852,0.25,Coconut Creek
4535,Richmond Heights,Richmond Heights,Richmond West,1139369881839607808,Neutro,0.814815,0.2,Richmond Heights
4652,Chicago,Chicago,Michigan/Chicago,1139372094259527680,Neutro,0.571429,0.3,Chicago
4718,Coconut Creek,Coconut Creek,Coconut Grove,1139373690053783552,Neutro,0.851852,0.25,Coconut Creek


In [193]:
ris=pd.merge(risultatiMigliori,legendaUS,on="city",how="inner")
update_df(ris)
len(id_trovati)

178695

In [194]:
pos_noCityAll_noNan=pos_noCityAll_noNan[~pos_noCityAll_noNan["id"].isin(id_trovati)]
pos_noCityAll_noNan.head(20)

,id,place_user,info1,info2,info3,info4,team,city
11,1139321659163201536,"Tipoca City, Kamino",Tipoca City,Kamino,nan,nan,Golden State Warriors,None
16,1139321690050244609,"ÜT: 41.886177,-87.601774",Üt: 41.886177,-87.601774,nan,nan,Golden State Warriors,None
31,1139321822220963841,"Banga, Western Visayas",Banga,Western Visayas,nan,nan,Golden State Warriors,Banga
32,1139321835743375360,"Silly Valley, CA",Silly Valley,CA,nan,nan,Golden State Warriors,None
38,1139321865975943168,"Bay Area, Califorina",Bay Area,Califorina,nan,nan,Golden State Warriors,None
43,1139321917150658560,"Valenzuela City, National Capi",Valenzuela City,National Capi,nan,nan,Golden State Warriors,None
59,1139322022754803712,"Scotland, United Kingdom",Scotland,United Kingdom,nan,nan,Golden State Warriors,None
72,1139322136516935680,"Parts, Unknown",Parts,Unknown,nan,nan,Golden State Warriors,None
83,1139322327982718977,"metro manila, philippines",Metro Manila,philippines,nan,nan,Golden State Warriors,None
100,1139322500293091328,"Republic of Slovenia , Serbia",Republic Of Slovenia,Serbia,nan,nan,Golden State Warriors,None


In [195]:
ris=pd.merge(pos_noCityAll_noNan,stati_US,left_on="info2",right_on="state_name",how="inner")
ris=ris[["info1","id","team","state_name","iso3"]]
ris.columns=["city","id","team","state_name","iso3"]
len(ris)

389

In [196]:
update_df(ris)
len(id_trovati)

179084

In [197]:
pos_noCityAll_noNan=pos_noCityAll_noNan[~pos_noCityAll_noNan["id"].isin(id_trovati)]
len(pos_noCityAll_noNan)

17727

In [198]:
ris=pd.merge(pos_noCityAll_noNan,stati_US,left_on="info2",right_on="state_id",how="inner")
ris=ris[["id","info1","team","state_id","state_name","iso3"]]
len(ris)

1591

In [199]:
ris_Simil=ris.apply(lambda x: estraiCitta_Simil_nofor(x['info1'],x['state_id'],x["id"],x["team"]),axis=1)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [200]:
ris_Simil=ottieni_dfpulitodafunzione(ris_Simil,ris)

In [201]:
ris=ris_Simil

In [202]:
risultatiMigliori=ris[ris["best_jacc"]==ris["best_edit"]]
risultatiMigliori=risultatiMigliori[risultatiMigliori["val_jacc"]<=0.3]
risultatiMigliori["city"]=risultatiMigliori["best_jacc"]
risultatiMigliori.head()

,best_jacc,val_jacc,best_edit,val_edit,city_originale,id,team,city


In [203]:
ris=pd.merge(risultatiMigliori,legendaUS,on="city",how="inner")
update_df(ris)
len(id_trovati)

179084

In [204]:
mancanti=gara_tweet[~gara_tweet["id"].isin(id_trovati)]

In [205]:
len(mancanti)

191336

In [206]:
mancanti_noNan=mancanti[(mancanti["place_user"]!="nan") & (mancanti["info2"]!="nan")]
mancanti_noNan.head(30)

,id,place_user,info1,info2,info3,info4,team,city
11,1139321659163201536,"Tipoca City, Kamino",Tipoca City,Kamino,nan,nan,Golden State Warriors,None
16,1139321690050244609,"ÜT: 41.886177,-87.601774",Üt: 41.886177,-87.601774,nan,nan,Golden State Warriors,None
31,1139321822220963841,"Banga, Western Visayas",Banga,Western Visayas,nan,nan,Golden State Warriors,Banga
32,1139321835743375360,"Silly Valley, CA",Silly Valley,CA,nan,nan,Golden State Warriors,None
38,1139321865975943168,"Bay Area, Califorina",Bay Area,Califorina,nan,nan,Golden State Warriors,None
43,1139321917150658560,"Valenzuela City, National Capi",Valenzuela City,National Capi,nan,nan,Golden State Warriors,None
59,1139322022754803712,"Scotland, United Kingdom",Scotland,United Kingdom,nan,nan,Golden State Warriors,None
72,1139322136516935680,"Parts, Unknown",Parts,Unknown,nan,nan,Golden State Warriors,None
83,1139322327982718977,"metro manila, philippines",Metro Manila,philippines,nan,nan,Golden State Warriors,None
100,1139322500293091328,"Republic of Slovenia , Serbia",Republic Of Slovenia,Serbia,nan,nan,Golden State Warriors,None


# Ultimo join tra info2 e country db stati_code (iso3 db)

In [212]:
ris=pd.merge(mancanti_noNan,stati_code_noUSACAN,left_on="info2",right_on="Country",how="inner")
len(ris)

2082

In [213]:
mancanti_noNan_info3=mancanti_noNan[mancanti_noNan["info3"]!="nan"]
len(mancanti_noNan_info3)

1046

# numero tweet finali matchati

In [279]:
len(gara_tweet[gara_tweet["id"].isin(id_trovati)])

89634

# Codice da eseguire per ultimo

In [215]:
df_infoagg["gara"]=6

In [216]:
len(gara_tweet[gara_tweet["place_user"]=="nan"])

92265

In [217]:
df_infoagg_country=pd.merge(df_infoagg,stati_code,on="iso3",how="inner")

In [218]:
len(df_infoagg)

179084

In [219]:
df_infoagg_country.to_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\gara6\\infoMapCountry_gara6.csv",
                 index=False)

# Concateno posizione tweet gara ottenuti prima con quelli già matchati

In [20]:
finale_match=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\infoMapCountry_final_v1.csv",
                 engine="python",encoding="utf-8")

In [286]:
finale_match.tail()

,Country,city,gara,id,iso2,iso3,state_name,team
662923,Hong Kong,Hong Kong,6,1139392841472364544,HK,HKG,NaN,Toronto Raptors
662924,Hong Kong,Hong Kong,6,1139393340221210624,HK,HKG,NaN,Toronto Raptors
662925,Hong Kong,Hong Kong,6,1139393760050069505,HK,HKG,NaN,Toronto Raptors
662926,Equatorial Guinea,Calatrava,6,1139378970699288576,GQ,GNQ,NaN,Neutro
662927,Yemen,Rida,6,1139385341654138881,YE,YEM,Al Bayḑā’,Toronto Raptors


In [222]:
finale_match=pd.concat([finale_match,df_infoagg_country],axis=0)
finale_match.tail()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,Country,city,gara,id,iso2,iso3,state_name,team
180099,Hong Kong,Hong Kong,6,1139392841472364544,HK,HKG,NaN,Toronto Raptors
180100,Hong Kong,Hong Kong,6,1139393340221210624,HK,HKG,NaN,Toronto Raptors
180101,Hong Kong,Hong Kong,6,1139393760050069505,HK,HKG,NaN,Toronto Raptors
180102,Equatorial Guinea,Calatrava,6,1139378970699288576,GQ,GNQ,NaN,Neutro
180103,Yemen,Rida,6,1139385341654138881,YE,YEM,Al Bayḑā’,Toronto Raptors


In [223]:
finale_match.to_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\posizioneTweet\\infoMapCountry_final_v1.csv",
                 index=False)

# Creo file dove per ogni città definisco quale è il tipo di tweet (GS, TOR ,NEUTRO) con più occorrenze

In [21]:
finale_match.head()

,Country,city,gara,id,iso2,iso3,state_name,team
0,United States,Washington,1,1134248146668851200,NaN,USA,District of Columbia,Golden State Warriors
1,United States,Washington,1,1134250383830896640,NaN,USA,District of Columbia,Golden State Warriors
2,United States,Washington,1,1134262039336161280,NaN,USA,District of Columbia,Golden State Warriors
3,United States,Washington,1,1134262960199675904,NaN,USA,District of Columbia,Golden State Warriors
4,United States,Washington,1,1134263268544004104,NaN,USA,District of Columbia,Golden State Warriors


In [23]:
def estraiNumTweetTeam(city,state,iso3,gara):
    dict={}
    df_conta=ris_numTweetTeam[(ris_numTweetTeam["city"]==city) & (ris_numTweetTeam["state_name"]==state) & 
                              (ris_numTweetTeam["iso3"]==iso3) & (ris_numTweetTeam["gara"]==gara)]
    for i in range(len(df_conta)):
        #print(df_conta.iloc[i]["team"]+": "+str(df_conta.iloc[i]["id"]))
        dict.update({df_conta.iloc[i]["team"]: df_conta.iloc[i]["id"]})
    return dict

In [24]:
ris_numTweetTeam=finale_match.groupby(["city","state_name","iso3","team","gara"])["id"].count().to_frame()
ris_numTweetTeam=ris_numTweetTeam.reset_index()

In [25]:
ris_dict=ris_numTweetTeam[["city","state_name","iso3","gara"]].drop_duplicates()
ris_dict["numTweetTeam"]=ris_dict.apply(lambda x:estraiNumTweetTeam(x["city"],x["state_name"],x["iso3"],x["gara"]),axis=1)
ris_split=ris_dict["numTweetTeam"].apply(pd.Series)
ris_gara=pd.merge(ris_dict,ris_split,left_index=True,right_index=True)

In [26]:
team_Magg=ris_numTweetTeam.loc[ris_numTweetTeam.groupby(["city","state_name","iso3","gara"])["id"].idxmax()]

In [174]:
ris_finale=pd.merge(ris_gara,team_Magg,on=["city","state_name","iso3","gara"])
ris_finale=ris_finale[["city","state_name","iso3","gara","Neutro","Toronto Raptors","Golden State Warriors","team"]]
ris_finale["Neutro"]=ris_finale["Neutro"].fillna(0).astype('int64')
ris_finale["Toronto Raptors"]=ris_finale["Toronto Raptors"].fillna(0).astype('int64')
ris_finale["Golden State Warriors"]=ris_finale["Golden State Warriors"].fillna(0).astype('int64')
ris_finale.head()

In [177]:
ris_finale.to_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\csv_finali\\infoMap_tweetCity.csv",
                 index=False)

# Carico file creato precedentemente

In [3]:
ris_finale=pd.read_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\csv_finali\\infoMap_tweetCity.csv",
                      engine="python", encoding="utf-8")

In [6]:
ris_finale.columns=["city","state_name","iso3","gara","Neutro","Toronto Raptors","Golden State Warriors","tipotweet_Magg"]

In [63]:
def trovaTeamMagg(tor,gs):
    if (tor==gs):
        return "Pari"
    else:
        if (tor>gs):
            return "Toronto Raptors"
        else:
            return "Golden State Warriors"

In [20]:
#prova=ris_finale[ris_finale["city"]=="Washington"]
ris_finale["team_Magg"]=ris_finale.apply(lambda x: trovaTeamMagg(x["Toronto Raptors"],x["Golden State Warriors"]),axis=1)

In [21]:
ris_finale.head()

,city,state_name,iso3,gara,Neutro,Toronto Raptors,Golden State Warriors,tipotweet_Magg,team_Magg
0,$D,California,USA,5,1,0,0,Neutro,Pari
1,(Plx) Portage Lakes,Ohio,USA,1,2,0,0,Neutro,Pari
2,0-16,Ohio,USA,6,1,0,0,Neutro,Pari
3,00727,Puerto Rico,USA,4,2,0,0,Neutro,Pari
4,00727,Puerto Rico,USA,5,1,0,0,Neutro,Pari


In [30]:
ris_finale=pd.merge(ris_finale,stati_code,on="iso3",how="inner")
ris_finale=ris_finale[["city","state_name","iso3","Country_y","gara","Neutro","Toronto Raptors","Golden State Warriors","tipotweet_Magg","team_Magg"]]
ris_finale.columns=["city","state_name","iso3","Country","gara","Neutro","Toronto Raptors","Golden State Warriors","tipotweet_Magg","team_Magg"]

In [31]:
ris_finale.head()

,city,state_name,iso3,Country,gara,Neutro,Toronto Raptors,Golden State Warriors,tipotweet_Magg,team_Magg
0,$D,California,USA,United States,5,1,0,0,Neutro,Pari
1,(Plx) Portage Lakes,Ohio,USA,United States,1,2,0,0,Neutro,Pari
2,0-16,Ohio,USA,United States,6,1,0,0,Neutro,Pari
3,00727,Puerto Rico,USA,United States,4,2,0,0,Neutro,Pari
4,00727,Puerto Rico,USA,United States,5,1,0,0,Neutro,Pari


In [32]:
ris_finale.to_csv("D:\\Università\\Magistrale\\Data management\\Progetto\\analisi\\csv_finali\\infoMap_tweetCity.csv",
                 index=False)